# Collected rook errors 2022-12-08


In [1]:
import os
os.environ['ROOK_URL'] = 'http://rook.dkrz.de/wps'

from rooki import operators as ops

## Process error: subset_level_by_values got multiple values for argument level_values 

fc4aed08-74a5-11ed-95c3-fa163eaaf620 |  1830 | execute   | 1.0.0   | 2022-12-05 15:06:16.95578  | 2022-12-05 15:06:19.901915 | orchestrate | Process error: subset_level_by_values got multiple values for argument level_values 

```
drwx------ 3 tomcat tomcat 114 Dec  5 15:06 pywps_process_7kwbnw67

workflow:
<![CDATA[{\\\"inputs\\\": {\\\"ta\\\": [\\\"c3s-cmip6.CMIP.NOAA-GFDL.GFDL-ESM4.historical.r1i1p1f1.Amon.ta.gr1.v20190726\\\"]}, \\\"steps\\\": {\\\"subset_ta_1\\\": {\\\"run\\\": \\\"subset\\\", \\\"in\\\": {\\\"collection\\\": \\\"inputs/ta\\\", \\\"time\\\": \\\"1950/1999\\\", \\\"time_components\\\": \\\"year:1950,1951,1952,1953,1954,1955,1956,1957,1958,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999|month:jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec\\\", \\\"level\\\": \\\"10000\\\", \\\"area\\\": \\\"-157,49.2,-117.3,67.6\\\"}}}, \\\"outputs\\\": {\\\"output\\\": \\\"subset_ta_1/output\\\"}, \\\"doc\\\": \\\"workflow\\\"}]]>

  File "/usr/local/anaconda/envs/rook/lib/python3.10/site-packages/clisops/ops/subset.py", line 105, in _calculate
    result = subset_bbox(ds, **kwargs)
  File "/usr/local/anaconda/envs/rook/lib/python3.10/site-packages/clisops/core/subset.py", line 270, in func_checker
    return func(*args, **kwargs)
  File "/usr/local/anaconda/envs/rook/lib/python3.10/site-packages/clisops/core/subset.py", line 1295, in subset_bbox
    da = subset_level_by_values(da, level_values)
  File "/usr/local/anaconda/envs/rook/lib/python3.10/site-packages/clisops/core/subset.py", line 308, in func_checker
    return func(*args, **kwargs)
TypeError: subset_level_by_values() got multiple values for argument 'level_values'
```


### Reproduce error ...

In [2]:
subset = ops.Subset(
        ops.Input(
            'noaa', ['c3s-cmip6.CMIP.NOAA-GFDL.GFDL-ESM4.historical.r1i1p1f1.Amon.ta.gr1.v20190726']
        ),
        time="1950/1950",
        level='10000',
        area='-157,49.2,-117.3,67.6'
)

wf = subset

In [3]:
resp = wf.orchestrate()
resp.ok

 owslib.wps.WPSException : {'code': 'NoApplicableCode', 'locator': 'None', 'text': 'Process error: subset_level_by_values got multiple values for argument level_values'}


False

**It happens when you run `level` and `area` subsetting in one step** 

In [4]:
# time + area works

wf = ops.Subset(
        ops.Input(
            'noaa', ['c3s-cmip6.CMIP.NOAA-GFDL.GFDL-ESM4.historical.r1i1p1f1.Amon.ta.gr1.v20190726']
        ),
        time="1950/1950",
        # level='10000',
        area='-157,49.2,-117.3,67.6'
)

resp = wf.orchestrate()
resp.ok

True

In [5]:
# time + level works

wf = ops.Subset(
        ops.Input(
            'noaa', ['c3s-cmip6.CMIP.NOAA-GFDL.GFDL-ESM4.historical.r1i1p1f1.Amon.ta.gr1.v20190726']
        ),
        time="1950/1950",
        level='10000',
        # area='-157,49.2,-117.3,67.6'
)

resp = wf.orchestrate()
resp.ok

True

### Workaround ... run level and area subsetting in seperate steps

In [6]:
subset = ops.Subset(
        ops.Input(
            'noaa', ['c3s-cmip6.CMIP.NOAA-GFDL.GFDL-ESM4.historical.r1i1p1f1.Amon.ta.gr1.v20190726']
        ),
        time="1950/1950",    
)

subset = ops.Subset(
        subset,
        level='10000',        
)

subset = ops.Subset(
    subset,
    area='-157,49.2,-117.3,67.6'
)

wf = subset

In [7]:
resp = wf.orchestrate()
resp.ok

True

In [8]:
ds = resp.datasets()[0]
ds

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 19, lon: 32, plev: 1, time: 12)
Coordinates:
  * bnds       (bnds) float64 1.0 2.0
  * lat        (lat) float64 49.5 50.5 51.5 52.5 53.5 ... 64.5 65.5 66.5 67.5
  * lon        (lon) float64 203.1 204.4 205.6 206.9 ... 238.1 239.4 240.6 241.9
  * plev       (plev) float64 1e+04
  * time       (time) object 1950-01-16 12:00:00 ... 1950-12-16 12:00:00
Data variables:
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    ta         (time, plev, lat, lon) float32 ...
    time_bnds  (time, bnds) object ...
Attributes: (12/46)
    external_variables:     areacella
    history:                File was processed by fremetar (GFDL analog of CM...
    table_id:               Amon
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    ...                     ...
    title:                  NOAA GFDL GFDL-ESM4 model output prepared for CMI...
    tracking_id:            hdl:21.14100/3042a141-567a-4c4a-a049-ff6d9a699b89
    variable_id:            ta
    variant_info:           N/A
    references:             see further_info_url attribute
    variant_label:          r1i1p1f1

## Process error: Requested levels include some not found in the dataset: 100000.0 

 b4cd76ca-6d80-11ed-b031-fa163eaaf620 | 22396 | execute   | 1.0.0   | 2022-11-26 12:51:47.496774 | 2022-11-26 12:51:50.651248 | orchestrate | Process error: Requested levels include some not found in the dataset: 100000.0 
